# Computing GC Content

## Problem

The GC-content of a DNA string is given by the percentage of symbols in the string that are 'C' or 'G'. For example, the GC-content of "AGCTATAG" is 37.5%. Note that the reverse complement of any DNA string has the same GC-content.

DNA strings must be labeled when they are consolidated into a database. A commonly used method of string labeling is called FASTA format. In this format, the string is introduced by a line that begins with '>', followed by some labeling information. Subsequent lines contain the string itself; the first line to begin with '>' indicates the label of the next string.

In Rosalind's implementation, a string in FASTA format will be labeled by the ID "Rosalind_xxxx", where "xxxx" denotes a four-digit code between 0000 and 9999.

_Given_: At most 10 DNA strings in FASTA format (of length at most 1 kbp each).

_Return_: The ID of the string having the highest GC-content, followed by the GC-content of that string. Rosalind allows for a default error of 0.001 in all decimal answers unless otherwise stated; please see the note on absolute error below.

**Sample Dataset**

    >Rosalind_6404
    CCTGCGGAAGATCGGCACTAGAATAGCCAGAACCGTTTCTCTGAGGCTTCCGGCCTTCCC
    TCCCACTAATAATTCTGAGG
    >Rosalind_5959
    CCATCGGTAGCGCATCCTTAGTCCAATTAAGTCCCTATCCAGGCGCTCCGCCGAAGGTCT
    ATATCCATTTGTCAGCAGACACGC
    >Rosalind_0808
    CCACCCTCGTGGTATGGCTAGGCATTCAGGAACCGGAGAACGCTTCAGACCAGCCCGGAC
    TGGGAACCTGCGGGCAGTAGGTGGAAT
    
**Sample Output**

    Rosalind_0808
    60.919540

**_Note on Absolute Error_**

We say that a number $x$ is within an absolute error of $y$ to a correct solution if $x$ is within $y$ of the correct solution. For example, if an exact solution is 6.157892, then for $x$ to be within an absolute error of 0.001, we must have that $|x−6.157892|<0.001$, or $6.156892<x<6.158892$.

Error bounding is a vital practical tool because of the inherent round-off error in representing decimals in a computer, where only a finite number of decimal places are allotted to any number. After being compounded over a number of operations, this round-off error can become evident. As a result, rather than testing whether two numbers are equal with $x=z$, you may wish to simply verify that $|x−z|$ is very small.

The mathematical field of numerical analysis is devoted to rigorously studying the nature of computational approximation.

_________________________________
## Solution

In this notebook we present three different and simple approaches to compute the string with highes GC-content. The first and simpler involves iterating through the data whie computing the GC content and keeping a tab on the string with the max value. The other approach is to use a list of tuples or a dictionary, in case the GC content is something we want to use later. And finally, and as we've been doing in the last few notebooks we introduce an implementation using 'Biopython'.

We start by saving the sample input as a '.txt' file. Now, in this first approach, we open the raw file and as we iterate through the FASTA sequences we keep track of the maximum GC value and string we have seen, updating it when necessary.

In [1]:
f = open('sample.txt', 'r')
sample = f.read()
f.close()

# Initialize the max values
max_gc_val = 0    
max_gc_str = ''

for fasta in sample.split(">")[1:]:
    # Prepare the data, split into sequence and ID
    seq_frags = fasta.split("\n")
    seq_id = seq_frags[0]
    seq = ''.join(seq_frags[1:])
    # Compute GC content of sequence
    gc = 100.0 * (seq.count("G") + seq.count("C")) / len(seq)
    # Compare against the current max and update if necessary
    if gc > max_gc_val:
        max_gc_val, max_gc_str = gc, seq_id
        
print(max_gc_str)
print('%.6f' % max_gc_val)

Rosalind_0808
60.919540


The above approach is the fastest, since we only have to traverse the initial string once. But if we want a little more flexibility or perform other calculations based on GC content (maybe we also want to find the string with the minimum value, or we want all strings sorted based on GC counts), we can populate an array of tuples (seq_id, GC_count). Once we have that data structure we can sort based on the second item (GC count).

In [2]:
f = open('sample.txt', 'r')
sample = f.read()
f.close()

# Initialize the max values
data = []

for fasta in sample.split(">")[1:]:
    # Prepare the data, split into sequence and ID
    seq_frags = fasta.split("\n")
    seq_id = seq_frags[0]
    seq = ''.join(seq_frags[1:])
    # Compute GC content of sequence
    gc = 100.0 * (seq.count("G") + seq.count("C")) / len(seq)
    data.append((seq_id,gc))
    
sorted_gc = sorted(data, key=lambda x: -x[1])
print(sorted_gc[0][0])
print('%.6f' %sorted_gc[0][1])

Rosalind_0808
60.919540


This approach takes a bit more time since we have to loop through the string once and then sort, which usually takes _O_(nlogn). We can also substitute the array of tuples by a dictionary. Depending on the kind of work we are doing we can make the sequence IDs the keys and the GC counts the value, or the opposite. However the former is somewhat more challenging since then we will be looking for the key with the highest value, instead of the value of the highest key. We present both approaches below:

In [3]:
# Using IDs as keys

f = open('sample.txt', 'r')
sample = f.read()
f.close()

# Initialize the max values
data = {}

for fasta in sample.split(">")[1:]:
    # Prepare the data, split into sequence and ID
    seq_frags = fasta.split("\n")
    seq_id = seq_frags[0]
    seq = ''.join(seq_frags[1:])
    # Compute GC content of sequence
    gc = 100.0 * (seq.count("G") + seq.count("C")) / len(seq)
    data[seq_id] = gc

# We take advantage of the fact that turning both
# keys and values into list preserves ordering
max_gc = max(list(data.values()))
max_gc_index = list(data.values()).index(max_gc)
max_id = list(data.keys())[max_gc_index]

print(max_id)
print('%6f' %max_gc)

Rosalind_0808
60.919540


In [4]:
# Using CG counts as keys

f = open('sample.txt', 'r')
sample = f.read()
f.close()

# Initialize the max values
data = {}

for fasta in sample.split(">")[1:]:
    # Prepare the data, split into sequence and ID
    seq_frags = fasta.split("\n")
    seq_id = seq_frags[0]
    seq = ''.join(seq_frags[1:])
    # Compute GC content of sequence
    gc = 100.0 * (seq.count("G") + seq.count("C")) / len(seq)
    data[gc] = seq_id

# We take advantage of the fact that turning both
# keys and values into list preserves ordering
max_gc = max(data.keys())
max_id = data[max_gc]

print(max_id)
print('%6f' %max_gc)

Rosalind_0808
60.919540


Finally we turn to Biopython. After what we have seen so far of this module (built-in translation and transcription, reverse complement, etc.), it is not surprising that Biopython offers a gc count method. We will implement the first solution in Biopython

In [5]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna
from Bio.SeqUtils import GC

f = open('sample.txt', 'r')
max_gc = 0
max_id = ''
for seq in SeqIO.parse(f,'fasta'):
    gc = GC(seq.seq)
    if gc > max_gc:
        max_gc = gc
        max_id = seq.id

print(max_id)
print('%6f' %max_gc)

Rosalind_0808
60.919540
